In [49]:
import pandas as pd
import os
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader

# from skmultilearn.adapt import MLkNN
# from sklearn.model_selection import GridSearchCV

In [84]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        #Encoder
        self.enc1 = nn.Linear(in_features=875, out_features=1024) 
        self.enc2 = nn.Linear(in_features=1024, out_features=512)
        self.enc3 = nn.Linear(in_features=512, out_features=256)
        self.enc4 = nn.Linear(in_features=256, out_features=128)
        self.enc5 = nn.Linear(in_features=128, out_features=128)

        #Decoder 
        self.dec1 = nn.Linear(in_features=128, out_features=128)
        self.dec2 = nn.Linear(in_features=128, out_features=256)
        self.dec3 = nn.Linear(in_features=256, out_features=512)
        self.dec4 = nn.Linear(in_features=512, out_features=1024)
        self.dec5 = nn.Linear(in_features=1024, out_features=875)

    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc5(x))

        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.relu(self.dec5(x))

        return x

In [53]:
train_feature = pd.read_csv('train_features.csv',index_col=0,header=0)
train_drug = pd.read_csv('train_drug.csv',index_col=0,header=0)
train_target = pd.read_csv('train_targets_scored.csv',index_col=0,header=0)
train_targets_nonscored = pd.read_csv('train_targets_nonscored.csv',index_col=0,header=0)
test_features = pd.read_csv('test_features.csv',index_col=0,header=0)
train_feature.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [54]:
# ?pd.get_dummies
# train_feature = pd.get_dummies(train_feature, prefix=['cp_type', 'cp_dose'])

train_feature['cp_type'] = train_feature['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})

train_feature['cp_dose'] = train_feature['cp_dose'].map({'D1':0,'D2':1})

train_feature.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,24,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,0,72,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,0,48,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,0,48,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,0,72,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [55]:
train_feature.shape

(23814, 875)

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_feature, train_target, test_size=0.20, random_state=42)

In [86]:
traindataloader = torch.utils.data.DataLoader(torch.from_numpy(X_train.to_numpy().astype(np.float32)).to(device), 
                                              shuffle=True, 
                                              batch_size=8000)
testdataloader = torch.utils.data.DataLoader(torch.from_numpy(X_test.to_numpy().astype(np.float32)).to(device), 
                                             shuffle=True, 
                                             batch_size=4000)

In [87]:
Lr_Rate = 1e-4
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=Lr_Rate)
model = model.to(device)
print(model)


Autoencoder(
  (enc1): Linear(in_features=875, out_features=1024, bias=True)
  (enc2): Linear(in_features=1024, out_features=512, bias=True)
  (enc3): Linear(in_features=512, out_features=256, bias=True)
  (enc4): Linear(in_features=256, out_features=128, bias=True)
  (enc5): Linear(in_features=128, out_features=128, bias=True)
  (dec1): Linear(in_features=128, out_features=128, bias=True)
  (dec2): Linear(in_features=128, out_features=256, bias=True)
  (dec3): Linear(in_features=256, out_features=512, bias=True)
  (dec4): Linear(in_features=512, out_features=1024, bias=True)
  (dec5): Linear(in_features=1024, out_features=875, bias=True)
)


In [88]:
for epochs in range(1000):
    train_loss = 0
    epoch_loss = 0
    for trdata in traindataloader:
        optimizer.zero_grad()
        out = model(trdata)
        train_loss = criterion(out,trdata)
        epoch_loss += train_loss
        train_loss.backward()
        optimizer.step()
        
    if epochs % 100 == 0:
        print("Epoch = ", epochs, "||", "epoch_loss = ", epoch_loss.detach().cpu().numpy().tolist())

Epoch =  0 || epoch_loss =  14.605568885803223
Epoch =  100 || epoch_loss =  5.018951892852783
Epoch =  200 || epoch_loss =  4.878135681152344
Epoch =  300 || epoch_loss =  4.798249244689941
Epoch =  400 || epoch_loss =  4.769577980041504
Epoch =  500 || epoch_loss =  4.725870132446289
Epoch =  600 || epoch_loss =  4.622297763824463
Epoch =  700 || epoch_loss =  4.584126949310303
Epoch =  800 || epoch_loss =  4.580447196960449
Epoch =  900 || epoch_loss =  4.501741886138916


In [92]:
for n in model.parameters():
    print(n)

Parameter containing:
tensor([[ 0.0075,  0.0325, -0.0192,  ...,  0.0294,  0.0174, -0.0317],
        [ 0.0343,  0.0118,  0.0105,  ...,  0.0296, -0.0163,  0.0281],
        [-0.0178,  0.0308, -0.0330,  ..., -0.0129, -0.0091,  0.0051],
        ...,
        [ 0.0318,  0.0061, -0.0067,  ..., -0.0116,  0.0009,  0.0174],
        [ 0.0071,  0.0190, -0.0297,  ...,  0.0269,  0.0021,  0.0311],
        [-0.0240,  0.0379,  0.0070,  ..., -0.0300,  0.0047,  0.0002]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0009, -0.0130,  0.0289,  ..., -0.0265,  0.0026,  0.0209],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[-0.0072, -0.0256, -0.0144,  ..., -0.0235, -0.0291, -0.0195],
        [ 0.0309, -0.0039, -0.0199,  ...,  0.0189, -0.0091,  0.0114],
        [-0.0094,  0.0219, -0.0011,  ..., -0.0203, -0.0166,  0.0001],
        ...,
        [ 0.0101,  0.0133,  0.0317,  ...,  0.0114, -0.0216, -0.0187],
        [-0.0235,  0.0160, -0.0053,  ...,  0.0104,